In [ ]:
import os

import geopandas
import numpy
import pandas
import rasterio
from shapely.geometry import Polygon
from tqdm.notebook import tqdm

In [ ]:
# May need to change working directory - TODO load from config
# os.cwd('/path/to/processed_data/nbs')

# Terrestrial - by cell prototype

In [ ]:
cell_number = 124

In [ ]:
grid = geopandas.read_file('../grid_10km.gpkg').set_index('link_id')
cell = grid.loc[cell_number]
cell

In [ ]:
subcells = geopandas.read_file('../grid_1km.gpkg') \
    .query(f'fid == {cell_number}') \
    .explode(index_parts=False) \
    .reset_index(drop=True)
subcells

In [ ]:
df = geopandas.read_file(f'land_use_slope/extracts/jamaica_land_use_combined_with_sectors_and_slope__{cell_number-1}.gpkg')

In [ ]:
df = df[['Classify', 'LU_CODE', 'cell_index', 'slope_degrees', 'geometry']] \
  .rename(columns={'Classify': 'landuse_desc', 'LU_CODE': 'landuse_code'})
df

In [ ]:
def associate_raster(fname, df, key, band_number=1, cell_index_col="cell_index"):
    with rasterio.open(fname) as dataset:
        assert df.crs == dataset.crs, "Raster and vector CRS must match"
        band_data = dataset.read(band_number)
        flat_data = band_data.flatten()
        df[key] = df[cell_index_col].apply(lambda i: flat_data[i])
    return df

In [ ]:
with_elevation = associate_raster('land_use_slope/elevation.tif', df, 'elevation_m')
with_elevation.elevation_m = numpy.clip(with_elevation.elevation_m, -100, 2256)

In [ ]:
def slice_vector(slice_df, slice_cell):
    slice_df = slice_df \
        .cx[slice_cell.left:slice_cell.right, slice_cell.bottom:slice_cell.top] \
        .copy()
    slice_df.geometry = slice_df.geometry.intersection(slice_cell.geometry)
    return slice_df

def associate_vector(fname, data_df, cell, subcells, select_cols):
    vector_df = geopandas.read_file(fname) \
        [list(select_cols.keys()) + ['geometry']] \
        .explode(index_parts=False) \
        .rename(columns=select_cols) \
        .copy()
    vector_df = slice_vector(vector_df, cell)
    if vector_df.empty:
        for col in select_cols.values():
            data_df[col] = None
        return data_df
    
    chunks = []
    chunk_size = 100
    for lower in tqdm(range(0, len(data_df), chunk_size)):
        chunk = data_df[lower:lower+chunk_size].copy()
        try:
            chunk = chunk.overlay(vector_df, how='identity', keep_geom_type=True)
        except ValueError:
            # probably had nothing to overlay
            for col in select_cols.values():
                chunk[col] = None
        chunks.append(chunk)
#    for subcell in tqdm(subcells.itertuples(), total=len(subcells)):
#        df_chunk = slice_vector(data_df, subcell)
#        if df_chunk.empty:
#            continue
#        vector_chunk = slice_vector(vector_df, subcell)
#        if vector_chunk.empty:
#            chunks.append(df_chunk)
#        else:
#            chunk = df_chunk.overlay(vector_chunk, how='identity')
#            chunks.append(chunk)
        
    return pandas.concat(chunks)

In [ ]:
with_soils = associate_vector(
    'soils/nsmdb-soils.gpkg', with_elevation, cell, subcells,
    {'Type':'soil_type', 'Permeability Code': 'hydrologic_soil_group_code'})

In [ ]:
#with_erosion = associate_vector(
#    'Terrestrial\Soil erosion susceptibility\Erosion susceptibility by land cover\Soil erosion susceptibility and land use intersect.shp',
#    with_soils, cell, subcells
#    {'Classes':'erosion_susceptibility'})

In [ ]:
with_bauxite = associate_vector(
    'Terrestrial/Bauxite/nsmdb-bauxite_reserves.gpkg', with_soils, cell, subcells,
    {'COLOR': 'within_bauxite_area'})
with_bauxite.within_bauxite_area = ~with_bauxite.within_bauxite_area.isna()

In [ ]:
with_primary_forest = associate_vector(
    'Terrestrial\Forests\Forests buffered 100m\Primary forest buffered 100m.shp', with_bauxite, cell, subcells,
    {'LU_CODE': 'within_forest_100m'})
with_primary_forest.within_forest_100m = (with_primary_forest.within_forest_100m == 'PF')

In [ ]:
with_forest_reserves = associate_vector(
    'Protected Sites\Forest Reserves\Forest Reserves.shp', with_primary_forest, cell, subcells, 
    {'NAME':'forest_reserve_name'})
with_forest_reserves['within_forest_reserve'] = ~with_forest_reserves.forest_reserve_name.isna()

In [ ]:
protected_areas = geopandas.read_file('Protected Sites\Protected Areas\Protected areas.shp')

In [ ]:
# for layer in protected_areas.LAYER.unique():
#     protected_area_type = protected_areas[protected_areas.LAYER == layer].copy()
#     protected_area_type.to_file(f'Protected Sites\Protected Areas\protected_areas_{layer}.gpkg', index=False)

In [ ]:
with_protected = with_forest_reserves
for layer in protected_areas.LAYER.unique():
    with_protected = associate_vector(
        f'Protected Sites\Protected Areas\protected_areas_{layer}.gpkg', 
        with_protected, cell, subcells, 
        {'NAME':f'protected_area_{layer}_name'})
with_protected['is_protected'] = ~(
    with_protected.protected_area_GAME_RESERVES_name.isna() |
    with_protected.protected_area_NATIONAL_PARK_name.isna() |
    with_protected.protected_area_PROTECTED_AREA_name.isna() |
    with_protected.protected_area_MARINE_PARK_name.isna()
)
with_protected['is_proposed_protected'] = ~with_protected.protected_area_PROPOSED_PROTECTED_AREA_name.isna()

In [ ]:
with_major = associate_vector(
    'Terrestrial/Riparian NbS/Rivers buffered 50m/Major rivers/Major rivers buffered 50m.gpkg',
    with_protected, cell, subcells,
    {'OBJECTID_1': 'within_major_river_50m'})
with_major.within_major_river_50m = ~with_major.within_major_river_50m.isna()

In [ ]:
with_stream = associate_vector(
    'Terrestrial/Riparian NbS/Rivers buffered 50m/Large streams/Large steams buffered 50m.gpkg',
    with_major, cell, subcells,
    {'OBJECTID': 'within_large_stream_50m'})
with_stream.within_large_stream_50m = ~with_stream.within_large_stream_50m.isna()

In [ ]:
with_headwater = associate_vector(
    'Terrestrial/Riparian NbS/Rivers buffered 50m/Headwater streams/Headwater streams buffered 50m.gpkg',
    with_stream, cell, subcells,
    {'OBJECTID': 'within_headwater_stream_50m'})
with_headwater.within_headwater_stream_50m = ~with_headwater.within_headwater_stream_50m.isna()

# Marine

In [ ]:
def associate_vector_direct(vector_df, data_df):    
    select_cols = [c for c in vector_df.columns if c != 'geometry']
    chunks = []
    chunk_size = 10
    for lower in tqdm(range(0, len(data_df), chunk_size)):
        chunk = data_df[lower:lower+chunk_size].copy()
        try:
            chunk = chunk.overlay(vector_df, how='identity', keep_geom_type=True)
        except ValueError:
            # probably had nothing to overlay
            for col in select_cols:
                chunk[col] = None
        chunks.append(chunk)        
    return pandas.concat(chunks)

In [ ]:
marine_combined = geopandas.read_file('Marine/marine_grid_10km.gpkg') \
    [['geometry']]

In [ ]:
seagrass = geopandas.read_file('Marine/Baseline coastal ecosystems/Seagrass/Seagrass.shp') \
    [['geometry']]
seagrass['is_seagrass'] = True
seagrass = seagrass.dissolve().explode(index_parts=False)
seagrass.head()

In [ ]:
coral = geopandas.read_file('Marine/Baseline coastal ecosystems/Coral Reefs/Coral Reefs.shp') \
    [['TNC_L4L3', 'geometry']] \
    .rename(columns={'TNC_L4L3': 'coral_type'})
coral['is_coral'] = True
coral.head(2)

In [ ]:
mangrove = geopandas.read_file('Marine/Baseline coastal ecosystems/Mangroves (Forces of Nature)\mangroves.shp') \
    [['TYPE', 'geometry']] \
    .rename(columns={'TYPE': 'mangrove_type'})
mangrove['is_mangrove'] = True
mangrove.head(2)

In [ ]:
marine_combined = associate_vector_direct(seagrass, marine_combined)

In [ ]:
marine_combined = associate_vector_direct(coral, marine_combined)

In [ ]:
marine_combined = associate_vector_direct(mangrove, marine_combined)

In [ ]:
for buffer in [500]:
    seagrass_buffer = seagrass.copy().drop(columns='is_seagrass')
    seagrass_buffer.geometry = seagrass.buffer(buffer)
    seagrass_buffer[f'within_seagrass_{buffer}m'] = True
    seagrass_buffer = seagrass_buffer.dissolve().explode(index_parts=False)
    marine_combined = associate_vector_direct(seagrass_buffer, marine_combined)

In [ ]:
for buffer in [500]:
    coral_buffer = coral.copy().drop(columns=['is_coral', 'coral_type'])
    coral_buffer.geometry = coral.buffer(buffer)
    coral_buffer[f'within_coral_{buffer}m'] = True
    coral_buffer = coral_buffer.dissolve().explode(index_parts=False)
    marine_combined = associate_vector_direct(coral_buffer, marine_combined)

In [ ]:
for buffer in [500]:
    mangrove_buffer = mangrove.copy().drop(columns=['is_mangrove', 'mangrove_type'])
    mangrove_buffer.geometry = mangrove.buffer(buffer)
    mangrove_buffer[f'within_mangrove_{buffer}m'] = True
    mangrove_buffer = mangrove_buffer.dissolve().explode(index_parts=False)
    marine_combined = associate_vector_direct(mangrove_buffer, marine_combined)

In [ ]:
try:
    marine_combined = marine_combined.drop(columns=['left', 'top', 'right', 'bottom', 'id'])
except KeyError:
    pass
marine_combined.columns

In [ ]:
data_cols = [c for c in marine_combined.columns if c != 'geometry']
marine_combined = marine_combined.dropna(how='all', subset=data_cols)

In [ ]:
marine_combined.to_file('Marine/marine_combined.gpkg')